In [1]:
pip install ttach #Test Time Augmentation

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import ttach as tta
import os
import matplotlib.pyplot as plt
import torchvision.models as models
# This is for the progress bar.
from tqdm import tqdm

In [3]:
# 看看label文件长啥样
labels_dataframe = pd.read_csv('../input/classify-leaves/train.csv')
labels_dataframe.head(3)

,image,label
0,images/0.jpg,maclura_pomifera
1,images/1.jpg,maclura_pomifera
2,images/2.jpg,maclura_pomifera


In [4]:
# 提取出lable并统计类别个数 排序
leaves_labels = sorted(list(set(labels_dataframe['label']))) #set删掉重复的 sort排序
n_classes = len(leaves_labels) #类别长度
print(n_classes) 

176


In [5]:
# 把label转成对应的数字
class_to_num = dict(zip(leaves_labels, range(n_classes)))
# 再转换回来，方便最后预测的时候使用
num_to_class = {v : k for k, v in class_to_num.items()}

In [6]:
# 继承pytorch的dataset，创建自己的Data
class LeavesData(Dataset):
    def __init__(self, csv_path, file_path, mode='train', valid_ratio=0.2, resize_height=256, resize_width=256):
        """
        Args:
            csv_path (string): csv 文件路径
            img_path (string): 图像文件所在路径
            mode (string): 训练模式还是测试模式
            valid_ratio (float): 验证集比例
        """
        
        # 需要调整后的照片尺寸，我这里每张图片的大小尺寸不一致#
        self.resize_height = resize_height
        self.resize_width = resize_width

        self.file_path = file_path
        self.mode = mode

        # 读取 csv 文件
        # 利用pandas读取csv文件
        self.data_info = pd.read_csv(csv_path, header=None)  #header=None是去掉表头部分
        # 计算 length                                        #但是会把表头放进列表
        self.data_len = len(self.data_info.index) - 1       #所以需要-1
        self.train_len = int(self.data_len * (1 - valid_ratio))
        
        if mode == 'train':
            # 第一列包含图像文件的名称   例如images/0.jpg
            self.train_image = np.asarray(self.data_info.iloc[1:self.train_len, 0])  #self.data_info.iloc[1:,0]表示读取第一列，从第二行开始到train_len
            # 第二列是图像的 label       例如maclura_pomifera  （叶子种类）
            self.train_label = np.asarray(self.data_info.iloc[1:self.train_len, 1])
            self.image_arr = self.train_image 
            self.label_arr = self.train_label
        elif mode == 'valid':  #验证集可以防止过拟合 观察拟合结果
            self.valid_image = np.asarray(self.data_info.iloc[self.train_len:, 0])  
            self.valid_label = np.asarray(self.data_info.iloc[self.train_len:, 1])
            self.image_arr = self.valid_image
            self.label_arr = self.valid_label
        elif mode == 'test':
            self.test_image = np.asarray(self.data_info.iloc[1:, 0])
            self.image_arr = self.test_image
            
        self.real_len = len(self.image_arr)

        print('Finished reading the {} set of Leaves Dataset ({} samples found)'
              .format(mode, self.real_len))

    def __getitem__(self, index):
        # 从 image_arr中得到索引对应的文件名
        single_image_name = self.image_arr[index]  #self.image_arr[0]='images/0.jpg'

        # 读取图像文件
        img_as_img = Image.open(self.file_path + single_image_name)

        #如果需要将RGB三通道的图片转换成灰度图片可参考下面两行
#         if img_as_img.mode != 'L':
#             img_as_img = img_as_img.convert('L')

        #设置好需要转换的变量，还可以包括一系列的nomarlize等等操作
        if self.mode == 'train':                           #下面是图像增广
            transform = transforms.Compose([
                #transforms.Resize(300),
                #transforms.CenterCrop(224),
                transforms.Resize(224),
                transforms.RandomHorizontalFlip(p=0.5),   #随机水平翻转
                transforms.RandomVerticalFlip(p=0.5),     #除了水平竖直反转之外其他的处理方法貌似都会降低acc
                #transforms.RandomResizedCrop((224, 224), scale=(0.7, 1)),
                #transforms.RandomCrop((60, 120)), # 随机剪裁
                # transforms.ColorJitter(0.3, 0.3, 0.2), # 修改亮度、对比度和饱和度
                #transforms.RandomRotation(180), # 依degrees 随机旋转一定角度   10
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                # Normalize(mean, std)按通道进行标准化，即先减均值，再除以标准差std
                 ])
        else:
            # valid和test不做数据增强  只需要裁剪变成张量Tensor
            transform = transforms.Compose([
                transforms.Resize(224),
                #transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        
        img_as_img = transform(img_as_img)  #图像处理
        
        if self.mode == 'test':
            return img_as_img  #测试集只需要返回图像
        else: #训练以及测试有效性
            # 得到图像的 string label
            label = self.label_arr[index]   #例子self.label_arr[0] = maclura_pomifera
            # number label
            number_label = class_to_num[label] #查阅字典  将类型转换为数字

            return img_as_img, number_label  #返回每一个index对应的图片数据和对应的label

    def __len__(self):
         return self.real_len  #self.real_len = len(self.image_arr) 返回的是训练/验证/测试/图像的数量

In [7]:
#设置文件路径并得到数据集
train_path = '../input/classify-leaves/train.csv'
test_path = '../input/classify-leaves/test.csv'
# csv文件中已经images的路径了，因此这里只到上一级目录
img_path = '../input/classify-leaves/'

train_dataset = LeavesData(train_path, img_path, mode='train')
val_dataset = LeavesData(train_path, img_path, mode='valid')
test_dataset = LeavesData(test_path, img_path, mode='test')
print(train_dataset)
print(val_dataset)
print(test_dataset)

Finished reading the train set of Leaves Dataset (14681 samples found)
Finished reading the valid set of Leaves Dataset (3672 samples found)
Finished reading the test set of Leaves Dataset (8800 samples found)


In [8]:
# 定义data loader
train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset,
        batch_size=90,      
        shuffle=True,     #打开乱序  False
        num_workers=2
    )

val_loader = torch.utils.data.DataLoader(
        dataset=val_dataset,
        batch_size=90,   
        shuffle=True,    #打开乱序  False
        num_workers=2
    )
test_loader = torch.utils.data.DataLoader(
        dataset=test_dataset,
        batch_size=90, 
        shuffle=False,
        num_workers=2
    )

In [9]:
# GPU计算
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'

device = get_device()
print(device)

cuda


In [10]:
# 超参数
learning_rate = 1e-4   #1e-4
weight_decay = 1e-3
num_epoch = 50
beta = 1              #cutmix参数
model_path = './pre_res_model.ckpt' #保存中间模型数据，方便加载

In [11]:
#微调学习率 
def train_fine_tuning(net,learning_rate,param_group=True):
    if param_group:
        params_1x = [
            param for name, param in net.named_parameters()
            if name not in ["fc.weight", "fc.bias"]]
        optimizer = torch.optim.Adam([{
            'params': params_1x}, {
                'params': net.fc.parameters(),
                'lr': learning_rate * 10}], lr=learning_rate,  #10
                                    weight_decay=0.001)
    else:
        optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate* 0.2,
                                      weight_decay=0.001)  
    return optimizer

In [12]:
!pip install timm
!pip install torchinfo
import timm                     #timm库有更丰富的预训练模型
from torchinfo import summary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.7/548.7 kB 9.0 MB/s eta 0:00:00a 0:00:01


In [13]:
model_1 = timm.create_model('seresnext50_32x4d', pretrained=True)
model_1.fc = nn.Linear(model_1.fc.in_features, 176)
nn.init.xavier_uniform_(model_1.fc.weight);
model_1 = model_1.to(device) #GPU
model_1.device = device

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/seresnext50_32x4d_racm-a304a460.pth" to /root/.cache/torch/hub/checkpoints/seresnext50_32x4d_racm-a304a460.pth


In [14]:
# model_2 = models.resnext50_32x4d(pretrained=True)
# model_2.fc = nn.Linear(model_2.fc.in_features, 176)
# nn.init.xavier_uniform_(model_2.fc.weight);
# model_2 = model_2.to(device) #GPU
# model_2.device = device

In [15]:
# model_3 = models.resnext50_32x4d(pretrained=True)
# model_3.fc = nn.Linear(model_3.fc.in_features, 176)
# nn.init.xavier_uniform_(model_3.fc.weight);
# model_3 = model_3.to(device) #GPU
# model_3.device = device

In [16]:
#cutmix计算裁剪区域
def rand_bbox(size, lamb):  #计算自定义裁剪区域
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lamb)
    cut_w = np.int32(W * cut_rat)
    cut_h = np.int32(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [17]:
#训练函数
def train_2(model): 
    optimizer = train_fine_tuning(model, learning_rate)
    # 对于分类任务，我们使用交叉熵作为性能的度量。
    criterion = nn.CrossEntropyLoss()
    # 定义学习率衰减
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1) #按照epoch衰减  不好用
    #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=40,eta_min=0.00000001)  #余弦变化
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True, min_lr=0.0000001)
    # 初始化优化器，您可以自行微调一些超参数，如学习速率。此处用的Adam
    #optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay=weight_decay)
    
    # 训练次数
    n_epochs = num_epoch
    
    best_acc = 0.0
    for epoch in range(n_epochs):
        # ---------- Training ----------  以下是训练模型
        # 训练前确保模型处于训练模式。
        model.train() 
        # 记录训练信息
        train_loss = []
        train_accs = []
        # 按批迭代训练集。
        for batch in tqdm(train_loader):
            # batch由图像数据和相应的标签组成。
            imgs, labels = batch
            imgs = imgs.to(device)    #数据移动到GPU
            labels = labels.to(device)
            
            #图片裁剪 CUTMIX训练代码
            lam = np.random.beta(beta, beta) #生成随机裁剪权重
            rand_index = torch.randperm(imgs.size()[0]).to(device) #打乱样本生成拼接样本
            labels_a = labels  #正常样本标签
            labels_b = labels[rand_index]  #乱序样本标签
            bbx1, bby1, bbx2, bby2 = rand_bbox(imgs.size(), lam) #生成裁剪区域
            #将原样本中bbx1:bbx2, bby1:bby2区域改成乱序样本标签对应的区域
            imgs[:, :, bbx1:bbx2, bby1:bby2] = imgs[rand_index, :, bbx1:bbx2, bby1:bby2]
            #重新计算lambda以精确匹配像素比率（因为有可能裁剪超出边界）
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (imgs.size()[-1] * imgs.size()[-2]))
            # Forward the data. (Make sure data and model are on the same device.)
            logits = model(imgs)  #将图形数据带入模型计算预测
            # 计算交叉熵损失(注意是两个样本的损失按照分割比例加权求和)
            loss = criterion(logits, labels_a) * lam + criterion(logits, labels_b) * (1. - lam)
            
            
            # 清除上一步中存储在参数中的梯度。
            optimizer.zero_grad()
            # 计算参数的梯度。
            loss.backward()
            # 用计算的梯度更新参数。
            optimizer.step()
        
            # 计算当前批次的精度。
            acc = (logits.argmax(dim=-1) == labels).float().mean()
    
            # 记录损失和准确度
            train_loss.append(loss.item())
            train_accs.append(acc)
            
                
            
        # 训练集的平均损失和精度是记录值的平均值。
        train_loss = sum(train_loss) / len(train_loss)
        train_acc = sum(train_accs) / len(train_accs)
        
        
        #更新学习率
        print("第%d个epoch的学习率：%f" % (epoch, optimizer.param_groups[0]['lr']))
        scheduler.step(train_loss)
        
        
        # 打印信息
        print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
            
            
        # ---------- 验证 ----------
        #这里我认为在验证处也加上TTA也许更加合理，因为前面对图像进行了变换，
        #在验证的时候也将图像进行相对应的变换可能能够提取更多的特征。
        # 确保模型处于eval模式，以便禁用dropout等模块并正常工作。
        model.eval()
        # 这些用于记录验证中的信息
        valid_loss = []
        valid_accs = []
        
        # 逐批迭代验证集。
        for batch in tqdm(val_loader):
            imgs, labels = batch
            # 不需要梯度验证.
            # Using  torch.no_grad()  accelerates the forward process.
            with torch.no_grad():
                logits = model(imgs.to(device))
                  
            # 我们仍然可以计算损失（但不能计算梯度）。
            loss = criterion(logits, labels.to(device))
    
            # 计算当前批次的精度。
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
    
            # 记录损失和准确性
            valid_loss.append(loss.item())
            valid_accs.append(acc)
              
        # 整个验证集的平均损失和准确度是记录值的平均值
        valid_loss = sum(valid_loss) / len(valid_loss)
        valid_acc = sum(valid_accs) / len(valid_accs)
    
        # 打印信息.
        print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
            
        # 如果模型改进了，在这个时间点保存一个检查点
        if valid_acc > best_acc:
            best_acc = valid_acc
            torch.save(model.state_dict(), model_path)
            print('saving model with acc {:.3f}'.format(best_acc))

In [18]:
saveFileName = './submission.csv'

In [19]:

train_2(model_1)
# model_1.load_state_dict(torch.load(model_path)) #加载训练结果
# train_2(model_2)
# model_2.load_state_dict(torch.load(model_path))
# train_2(model_3)
# model_3.load_state_dict(torch.load(model_path))

100%|██████████| 164/164 [02:07<00:00,  1.29it/s]


第0个epoch的学习率：0.000100
[ Train | 001/050 ] loss = 4.06971, acc = 0.18054


100%|██████████| 41/41 [00:21<00:00,  1.92it/s]


[ Valid | 001/050 ] loss = 1.50236, acc = 0.64743
saving model with acc 0.647


100%|██████████| 164/164 [01:59<00:00,  1.37it/s]


...

[ Valid | 049/050 ] loss = 0.13409, acc = 0.97798


100%|██████████| 164/164 [02:00<00:00,  1.37it/s]


第49个epoch的学习率：0.000013
[ Train | 050/050 ] loss = 0.93081, acc = 0.69088


100%|██████████| 41/41 [00:11<00:00,  3.44it/s]

[ Valid | 050/050 ] loss = 0.12873, acc = 0.97873
